In [1]:
import re, os
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./data/preprocessed/DebateNet-migr15_all_lemmatized.tsv", sep="\t", encoding="utf-8")
df

,source,date,quote,quote_lemmatized,values,values_minimal
0,1,20151215,"Amnesty International bezweifelt, dass eine s...",Amnesty International bezweifeln Vorgeleg ru...,"['800', '801']",{'8'}
1,1,20150126,"Gefordert werden ein Burka-Verbot, die strikte...",fordern Burka-Verbot strikt Begrenzung Einwand...,"['100', '102', '200', '207', '800', '812']","{'8', '2', '1'}"
2,1,20150225,"Wenn der Staat das nicht will, soll er seine A...",Staat sollen Arbeit gut Mensch Schutz gewähren,"['400', '402']",{'4'}
3,1,20150122,"Zweitens heißt es, dem Rassismus und der Demok...",zweitens heißen Rassismus Demokratieverachtung...,"['700', '706']",{'7'}
4,1,20151007,Laut Gesetz müssen sie aber zunächst in einer ...,laut Gesetz müssen zunächst Erstaufnahmeeinric...,"['200', '201']",{'2'}
...,...,...,...,...,...,...
3291,2,20151009,Die hat nun ausdrücklich die SPD ins Spiel geb...,ausdrücklich SPD Spiel bringen Vizekanzler Sig...,"['100', '102']",{'1'}
3292,2,20150913,Die Regierungschefs von Rheinland-Pfalz und Sc...,Regierungschef Rheinland-Pfalz Schleswig-Holst...,"['800', '808']",{'8'}
3293,2,20150202,Unterstützung erhielt er jetzt aber vom Wirtsc...,Unterstützung erhalten Wirtschaftsflügel Partei,"['100', '108']",{'1'}
3294,2,20150618,Neben Einmal-Zuschüssen will das Ministerium d...,neben Einmal-Zuschüss Ministerium Kommune dadu...,"['800', '803', '805']",{'8'}


In [3]:
all_quotes = []
all_labels = []

for idx, row in df.iterrows():
    current_labels = row["values_minimal"].strip('{}').split(', ')
    current_labels = [int(l.strip("'")) for l in current_labels]
    
    for label in current_labels:
        all_quotes.append(row["quote_lemmatized"])
        all_labels.append(label)
        
df_quote_label_mapping = pd.DataFrame({"text": all_quotes, "label": all_labels})
print(df_quote_label_mapping.value_counts("label"))
df_quote_label_mapping

label
1    864
7    590
5    572
8    562
2    531
3    326
4    154
6    144
9     26
Name: count, dtype: int64


,text,label
0,Amnesty International bezweifeln Vorgeleg ru...,8
1,fordern Burka-Verbot strikt Begrenzung Einwand...,8
2,fordern Burka-Verbot strikt Begrenzung Einwand...,2
3,fordern Burka-Verbot strikt Begrenzung Einwand...,1
4,Staat sollen Arbeit gut Mensch Schutz gewähren,4
...,...,...
3764,ausdrücklich SPD Spiel bringen Vizekanzler Sig...,1
3765,Regierungschef Rheinland-Pfalz Schleswig-Holst...,8
3766,Unterstützung erhalten Wirtschaftsflügel Partei,1
3767,neben Einmal-Zuschüss Ministerium Kommune dadu...,8


In [4]:
all_tokens = []
all_labels = []

for idx, row in df_quote_label_mapping.iterrows():
    current_tokens = row["text"].split()
    
    for token in current_tokens:
        all_tokens.append(token)
        all_labels.append(row["label"])
        
df_token_label_mapping = pd.DataFrame({"token": all_tokens, "label": all_labels})
print(df_token_label_mapping.value_counts("label"))
df_token_label_mapping

label
1    8307
5    5793
7    5792
8    5583
2    5185
3    3282
6    1445
4    1413
9     211
Name: count, dtype: int64


,token,label
0,Amnesty,8
1,International,8
2,bezweifeln,8
3,Vorgeleg,8
4,rufen,8
...,...,...
37006,endlich,8
37007,strukturell,8
37008,nachhaltig,8
37009,Ausgabe,8


In [5]:
def pmi(dff, x, y):
    df = dff.copy()
    df['f_x'] = df.groupby(x)[x].transform('count')
    df['f_y'] = df.groupby(y)[y].transform('count')
    df['f_xy'] = df.groupby([x, y])[x].transform('count')
    df['pmi'] = np.log(len(df.index) * df['f_xy'] / (df['f_x'] * df['f_y']) )
    df['pmi_freq'] = np.log(df["f_y"]) * np.log(len(df.index) * df['f_xy'] / (df['f_x'] * df['f_y']) ) 
    
    df.loc[df["f_y"] < 5 , "pmi_freq"] = 0
    
    return df

In [6]:
df_pmi = pmi(dff=df_token_label_mapping, x="label", y="token")
df_pmi

,token,label,f_x,f_y,f_xy,pmi,pmi_freq
0,Amnesty,8,5583,10,2,0.282051,0.649446
1,International,8,5583,10,2,0.282051,0.649446
2,bezweifeln,8,5583,3,2,1.486024,0.000000
3,Vorgeleg,8,5583,3,2,1.486024,0.000000
4,rufen,8,5583,33,4,-0.218724,-0.764771
...,...,...,...,...,...,...,...
37006,endlich,8,5583,37,11,0.678466,2.449886
37007,strukturell,8,5583,5,3,1.380663,2.222092
37008,nachhaltig,8,5583,7,3,1.044191,2.031902
37009,Ausgabe,8,5583,7,5,1.555017,3.025923


In [7]:
for i in range(1, 10):
    df_keywords = df_pmi[df_pmi["label"] == i].drop_duplicates().sort_values(by='pmi_freq', ascending=False)
    df_keywords = df_keywords.iloc[:200, ]
    df_keywords.to_csv(f"./output/label_{str(i)}.tsv", sep="\t", encoding="utf-8", index=False)

OSError: Cannot save file into a non-existent directory: 'raw_output'